# Text Analysis on ebola-essien data

In [3]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\ebola-essien.csv', encoding = 'utf-8', header = None)
df.columns = ['src_tweet_id', 'src_user_id', 'src_tweet','src_date','reply_tweet_id','reply_user_id','reply_tweet','reply_date', 'label']
df.reply_tweet = df.reply_tweet.replace(np.nan,"")

In [5]:
src_tw_df = df.iloc[:,[0,1,2,3,8]].copy()
src_tw_df = src_tw_df.drop_duplicates()
src_tw_df

,src_tweet_id,src_user_id,src_tweet,src_date,label
0,521310417696858112,964926744,Breaking news: Ghana international and AC Mila...,Sun Oct 12 14:44:23 +0000 2014,1
26,521346721226711040,579635019,Unconfirmed reports claim that Michael Essien ...,Sun Oct 12 17:08:39 +0000 2014,1
52,521350916872876033,2449736654,BREAKING: Unconfirmed reports claim AC Milan m...,Sun Oct 12 17:25:19 +0000 2014,1
70,521358118597689344,24044553,Milan have stated that the reports about Essie...,Sun Oct 12 17:53:56 +0000 2014,1
74,521359454672269313,2330299633,AC Milan midfielder Michael Essien has been di...,Sun Oct 12 17:59:15 +0000 2014,1
98,521359863884967936,315113964,Milan release official statement on their site...,Sun Oct 12 18:00:52 +0000 2014,1
107,521360486387175424,1594903015,AC Milan midfielder Michael Essien has been di...,Sun Oct 12 18:03:21 +0000 2014,1
115,521361857810939905,1742526258,AC Milan have confirmed that the reports about...,Sun Oct 12 18:08:47 +0000 2014,1
128,521367917322338304,1262229906,No truth in internet rumours that I have contr...,Sun Oct 12 18:32:52 +0000 2014,1
146,521369179392581632,159412087,Micheal Essien denying the Ebola rumours like ...,Sun Oct 12 18:37:53 +0000 2014,1


## Prepocessing
- Define function to process text at one time

In [6]:
def textprocessing(text):
    text = text.lower()

    text = " ".join([word for word in text.split() if 'http' not in word
                                and not word.startswith('@')])
                                #and word != 'RT'])
    import re
    def remove_punct(tweet):
        new_words = []
        for word in tweet:
            w = re.sub(r'[^\w\s]','',word) #remove everything except words and space
            w = re.sub(r'_','',w) #how to remove underscore as well
            new_words.append(w)

        return new_words
    text = "".join(remove_punct(text))
        
    from nltk.stem.porter import PorterStemmer
    stem = PorterStemmer()
    
    from nltk.tokenize import TweetTokenizer
    tknzr = TweetTokenizer(strip_handles=True)
    text = tknzr.tokenize(text)
    
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words("english"))
    def filterstopwords(tw):
        filter_stopwords = []
        for w in tw:
            if w not in stop_words:
                filter_stopwords.append(w)
        return filter_stopwords
    text = filterstopwords(text)
    text = " ".join(text)
    
    return text

In [7]:
print("original text: ", df.reply_tweet.iloc[20])
print("cleaned text: ", textprocessing(df.reply_tweet.iloc[20]))

original text:  Wow“@officialAnietie: what????? “@YuryAlkaev: Breaking news: Ghana international and AC Milan star Michael Essien has contracted Ebola, his
cleaned text:  wowofficialanietie yuryalkaev breaking news ghana international ac milan star michael essien contracted ebola


### applied on original data (**df**)

In [8]:
df['cleaned_src_tw'] = df.src_tweet.apply(textprocessing)
df['cleaned_reply_tw'] = df.reply_tweet.apply(textprocessing)

In [9]:
df.head()

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label,cleaned_src_tw,cleaned_reply_tw
0,521310417696858112,964926744,Breaking news: Ghana international and AC Mila...,Sun Oct 12 14:44:23 +0000 2014,521311862118711296,630241693,“@YuryAlkaev: Breaking news: Ghana internation...,Sun Oct 12 14:50:08 +0000 2014,1,breaking news ghana international ac milan sta...,yuryalkaev breaking news ghana international a...
1,521310417696858112,964926744,Breaking news: Ghana international and AC Mila...,Sun Oct 12 14:44:23 +0000 2014,521313155004497920,131531989,conspiracy RT@YuryAlkaev Breaking news: Ghana ...,Sun Oct 12 14:55:16 +0000 2014,1,breaking news ghana international ac milan sta...,conspiracy rtyuryalkaev breaking news ghana in...
2,521310417696858112,964926744,Breaking news: Ghana international and AC Mila...,Sun Oct 12 14:44:23 +0000 2014,521313840416714752,40673379,“@YuryAlkaev: Breaking news: Ghana internation...,Sun Oct 12 14:57:59 +0000 2014,1,breaking news ghana international ac milan sta...,yuryalkaev breaking news ghana international a...
3,521310417696858112,964926744,Breaking news: Ghana international and AC Mila...,Sun Oct 12 14:44:23 +0000 2014,521313949351178240,1628128982,“@YuryAlkaev: Breaking news: Ghana internation...,Sun Oct 12 14:58:25 +0000 2014,1,breaking news ghana international ac milan sta...,yuryalkaev breaking news ghana international a...
4,521310417696858112,964926744,Breaking news: Ghana international and AC Mila...,Sun Oct 12 14:44:23 +0000 2014,521314256911077376,349230142,“@YuryAlkaev: Breaking news: Ghana internation...,Sun Oct 12 14:59:39 +0000 2014,1,breaking news ghana international ac milan sta...,yuryalkaev breaking news ghana international a...


### applied on source tweet data (**src_tw_df**) 

In [10]:
src_tw_df['cleaned_src_tw'] = src_tw_df.src_tweet.apply(textprocessing)

## Sentiment Analysis

In [11]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [12]:
def get_sentiment(tweet):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(tweet)
    if vs['compound'] >=0.05:
        sentiment_label = 'Positive'
    elif (vs['compound'] > -0.05) & (vs['compound'] < 0.05):
        sentiment_label = 'Neutral'
    elif vs['compound']<= -0.05:
        sentiment_label = 'Negative' 
    result = sentiment_label
    return result

### applied on original df

In [13]:
df['src_sentiment']= df.cleaned_src_tw.apply(get_sentiment)
df['reply_sentiment']= df.cleaned_reply_tw.apply(get_sentiment)

In [ ]:
df.head(1)

### applied on source tweet data only 

In [14]:
### source tweet data only 
src_tw_df['src_sentiment']=  src_tw_df['cleaned_src_tw'].apply(get_sentiment)

### Visualization

In [ ]:
print(df[df.label ==1]['reply_sentiment'].value_counts())
plt.figure(figsize=(8,6))
sns.countplot(x='reply_sentiment',data= df[df['label'] == 1])

plt.suptitle('Sentiment analysis on Reply tweets under Catergory Rumour',fontsize=16)
plt.title('Reply tweets show more positive attitude towards rumour tweet',fontsize=12,color='grey')
plt.savefig('graph/ebola-essien/senti_reply_rumour.pdf',dpi=300)

In [ ]:
print(df[df.label ==0]['reply_sentiment'].value_counts())
plt.figure(figsize=(8,6))
sns.countplot(x='reply_sentiment',data= df[df['label'] == 0])

plt.suptitle('Sentiment analysis on Reply tweets under Catergory Non-Rumour',fontsize=16)
plt.title('Reply tweets show more neutrual attitude towards non-rumour tweet',fontsize=12,color='grey')
plt.savefig('graph/ebola-essien/senti_reply_nonrumour.pdf',dpi=300)

In [ ]:
print(src_tw_df['src_sentiment'].value_counts())
sns.countplot(x='src_sentiment',data =src_tw_df,hue='label')

plt.suptitle('Sentiment analysis on Source Tweets',fontsize=16)
plt.title('Rumour source tweets show more neutral attitude.',fontsize=12,color='grey')
plt.savefig('graph/ebola-essien/senti_source.pdf',dpi=300)

### save data to csv 

In [15]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\ebola-essien-df.csv',index=False)
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\ebola-essien-src.csv',index=False)

## Keywords Extraction
### WordCloud

In [ ]:
from wordcloud import WordCloud

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(docx)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()

#### Reply Tweets

In [ ]:
#nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [ ]:
reply_sen = df.cleaned_reply_tw.apply(word_tokenize)
reply_tw_list = []
for sen in reply_sen:
    for token in sen:
        reply_tw_list.append(token)
reply_tw_doc = " ".join(reply_tw_list)

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(reply_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/ebola-essien/wordcloud_reply.pdf')

In [ ]:
# positive 
posi_reply_sen = df[df.reply_sentiment=='Positive'].cleaned_reply_tw.apply(word_tokenize)
posi_reply_tw_list = []
for sen in posi_reply_sen:
    for token in sen:
        posi_reply_tw_list.append(token)
posi_reply_tw_doc = " ".join(posi_reply_tw_list)


In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(posi_reply_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/ebola-essien/wordcloud_reply_posi.pdf')

#### Source Tweet

In [ ]:
src_sen = src_tw_df.cleaned_src_tw.apply(word_tokenize)
src_tw_list = []
for sen in src_sen:
    for token in sen:
        src_tw_list.append(token)

src_tw_doc = " ".join(src_tw_list)

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(src_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/ebola-essien/wordcloud_src.pdf')

### Common

In [ ]:
from collections import Counter

In [ ]:
def get_tokens(docx,num=30):
    word_tokens = Counter(docx)
    most_common = word_tokens.most_common(num)
    result = dict(most_common)
    return result

In [ ]:
pd.DataFrame(get_tokens(reply_tw_list).items(),columns=['word','freq']).plot(kind='bar',x='word',y='freq')
plt.title("Most frequent words among reply tweets are 'Prince','show','tonight'.")

In [ ]:
pd.DataFrame(get_tokens(src_tw_list).items(),columns=['word','freq']).plot(kind='bar',x='word',y='freq')
plt.title("Most frequent words among source tweets are 'Price', 'show', 'tonight'")


## Emotion Detection

### text2emotion package
https://snyk.io/advisor/python/text2emotion 

In [16]:
import text2emotion as te

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
emotion = df.cleaned_reply_tw.apply(te.get_emotion)

In [18]:
emo = emotion.apply(lambda x: max(x,key=x.get))
df['emotion']=emo

In [24]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\ebola-essien-df.csv',index=False)

In [26]:
emotion1 = src_tw_df.cleaned_src_tw.apply(te.get_emotion)
emo1 = emotion.apply(lambda x: max(x,key=x.get))
src_tw_df['emotion']=emo1

In [27]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\ebola-essien-src.csv',index=False)